## TODO
class vehicle movement predictor
put vehicle to x0,y0,a0 and rotate all the points
receive vehicle north and host point


In [1]:
print('test')

test


## Cameras

In [2]:
from src.vision import Camera

mba_video_camera = Camera(
    h_fov=59.0,
    h_res=1080,
    v_res=720,
)

mba_photo_camera = Camera(
    h_fov=67.0,
    h_res=1290,
    v_res=720,
)

zed_two_camera = Camera(
    h_fov=110.0,
    v_fov=70.0,
)

avg_shoulder_width = 350.0
avg_torso_height = 460.0


In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import numpy as np
import pandas as pd
import cv2
import time

import torch

from src.vehicle import Vehicle
from src.vehicle import VehicleMover
from src.point import Point
from src.vision import FakeSensor
from src.point import create_map



In [ ]:
# VEHICLE
vehicle = Vehicle(
    max_turn_angle=30, 
    axle_len=265, 
    max_speed=2000, 
    camera=zed_two_camera,
    center=Point(0, 0)
)

# FAKE SENSOR
sensor_data = pd.read_csv('dist_2fps.csv', header=None)
sensor_data.columns = ['t', 'x', 'y']
sensor_data['t'] = sensor_data['t'].astype(float)
fake_sensor = FakeSensor(sensor_data, to_recenter=True)

# VEHICLE MOVER
vehicle_mover = VehicleMover(vehicle, fake_sensor, smoothing=4)

while not vehicle_mover.is_done():
    incoming, passed, last_seen, target = vehicle_mover.step()

    fig, ax = create_map(Point(0, 0), 6000, 6)
    ax.scatter(target.y, target.x, marker='x', color='b')
    ax.plot(incoming['y'], incoming['x'], label='Spline')
    ax.plot(passed['y'], passed['x'], label='Passed')
    Point(0, 0).scatter(ax, color='r', size=20)
    for x in range(len(vehicle_mover.seen)):
        Point.from_df(vehicle_mover.seen.iloc[x]).scatter(ax, color='k', size=20)
    
    ax.plot(vehicle_mover.actual_path['y'], vehicle_mover.actual_path['x'], label='Actual Path')

    fake_sensor.scatter_host(ax, vehicle)
    vehicle.plot(ax)
    ax.legend()
    plt.show()


# if the angle is more than agle max, back up and turn